## Install dependencies

In [ ]:
%%capture
!pip install tqdm vertexai sentencepiece matplotlib
!gcloud auth application-default login

## Put Data in JSON file

In [2]:

# import os
# import json

# # Directory containing the data files
# data_dir = "./data/middle"

# # Initialize an empty list to store data
# all_data = []

# # Loop through all files in the data directory
# for filename in os.listdir(data_dir):
#     if filename.endswith(".txt"):
#         # Read the JSON content of the file
#         with open(os.path.join(data_dir, filename), "r") as file:
#             file_data = json.load(file)
        
#         # Append the data to the list
#         all_data.append(file_data)

# # Write the data to a JSON file
# with open("middle.json", "w") as json_file:
#     json.dump(all_data, json_file, indent=4)


## Summarize paragraph -> Answer Questions

In [42]:
import json
from tqdm import tqdm
from vertexai.generative_models import GenerativeModel
from vertexai import generative_models

# Parameters
input_file = "middle.json"  # Initial data file
output_file = "results.json"  # Output file with summaries and answers
edit_query = '''Edit this paragraph to make it as concise as possible, capturing the main ideas clearly and briefly without losing information needed to answer the given questions. Provide the edited paragraph as-is without any additional context, labels, headings, or formatting. Do not include the answers: '''
answer_query = '''Based solely on the given context, answer the question with just a letter A-D corresponding to the most likely correct answer choice. Provide the response as-is without any additional context, labels, headings, or formatting: '''

# Load the JSON data from the file
with open(input_file, 'r') as file:
    data = json.load(file)

# Model setup
model = GenerativeModel("gemini-1.0-pro-002")
safety_config = {
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
}

# Process each item
output_data = []
limit = -1
for item in tqdm(data[:limit], desc="Processing"):
    # Generate concise summary
    edit_prompt = f"{edit_query}\nContext: {item['article']}\nQuestions:\n" + "\n".join(item['questions'])
    try:
        edit_response = model.generate_content(edit_prompt, generation_config={"max_output_tokens": 5000, "temperature": 0, "top_p": 1.0}, safety_settings=safety_config)
        summary = edit_response.text
    except Exception as e:
        print(f"Error processing item with ID {item['id']} during summarization: {str(e)}")
        summary = "Error"
    
    # Generate answers based on summary
    model_answers = []
    for question, options in zip(item['questions'], item['options']):
        options_text = '\n'.join(f"{chr(65+i)}. {opt}" for i, opt in enumerate(options))
        answer_prompt = f"{answer_query}\nContext: {summary}\nQuestion: {question}\n{options_text}\nAnswer:"
        try:
            answer_response = model.generate_content(answer_prompt, generation_config={"max_output_tokens": 1, "temperature": 0, "top_p": 1.0}, safety_settings=safety_config)
            model_answers.append(answer_response.text.strip())
        except Exception as e:
            print(f"Error processing item with ID {item['id']} during answering: {str(e)}")
            model_answers.append("Error")

    # Store the processed data
    item['summary'] = summary
    item['model_answers'] = model_answers
    output_data.append(item)

# Write the updated data to a JSON file
with open(output_file, 'w') as file:
    json.dump(output_data, file, indent=2)


Processing:   5%|▍         | 294/6408 [14:41<6:22:53,  3.76s/it]

Error processing item with ID middle1356.txt during answering: 500 Internal error encountered.


Processing:   6%|▌         | 388/6408 [19:26<4:38:23,  2.77s/it]

Error processing item with ID middle1468.txt during answering: Response has no candidates (and no text).


Processing:   6%|▋         | 403/6408 [20:14<5:01:29,  3.01s/it]


KeyboardInterrupt: 

## Visualize results


In [ ]:
# Load the JSON data from the processed file
file = 'results.json'  # Adjust the path as needed
with open(file, 'r') as f:
    data = json.load(f)

total_questions = 0
correct_answers = 0

# Iterate through each entry in the JSON data
for item in data:
    # Retrieve the list of correct answers and model-generated answers
    correct = item['answers']
    model_generated = item['model_answers']

    # Increment the total question count
    total_questions += len(correct)

    # Compare each answer with the model answer and count correct ones
    correct_answers += sum(1 for correct_answer, model_answer in zip(correct, model_generated) if correct_answer == model_answer)

# Calculate the accuracy
accuracy = (correct_answers / total_questions) * 100 if total_questions > 0 else 0

# Output the results
print(f'Total questions: {total_questions}')
print(f'Correct answers: {correct_answers}')
print(f'Accuracy: {accuracy:.2f}%')

Total questions: 17
Correct answers: 14
Accuracy: 82.35%
